### WGKB Gene Database Query

#### Reading data via web scraping

In [ ]:
import requests

In [ ]:
url = 'https://www.ncbi.nlm.nih.gov/datasets/gene/GCF_001411555.2/'

payload = {
    'accession': 'GCF_001411555.2',
    'gene_types': [],
    'locations': [],
    'page_size': 20,
    'page_token': '',
    'search_text': [],
    'sort': []
}

headers = {
    'Origin': 'https://www.ncbi.nlm.nih.gov',
    'Referer': 'https://www.ncbi.nlm.nih.gov/datasets/gene/GCF_001411555.2/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}

result = requests.post(url, headers=headers, params=payload)

#### Reading data via REST API

In [3]:
import requests

In [12]:
url = 'https://api.ncbi.nlm.nih.gov/datasets/v2alpha'

taxon = 'Juglans regia (English walnut)'
api_key = 'removed'

params = {
    'name': taxon,
    'api_key': api_key
}

response = requests.get(url, params=params)

In [13]:
response.text

# Might have to enable the NCBI API key. Don't think that will be viable if it's tied to my account.

'404 page not found\n'

#### Reading metadata with NCBI documented APIs (GenomeAPI, , GeneAPI, APIClient)

In [4]:
# !pip install ncbi-datasets-pylib

In [1]:
from ncbi.datasets import GenomeApi, GeneApi
from ncbi.datasets.openapi import ApiClient

##### Get chromosome metadata

In [3]:
def get_chrom_locations(organism_name: str) -> list:
    client = ApiClient()
    genome_api = GenomeApi(client)

    # Search for genome assemblies by organism name
    genome_summaries = genome_api.assembly_descriptors_by_taxon(organism_name)
    genomic_locations = []

    if genome_summaries.assemblies:
        for chrom_dict in genome_summaries.assemblies[0]['assembly']['chromosomes']:
            genomic_locations.append({
                'RefSeq': chrom_dict.accession_version,
                'Size': chrom_dict.length,
                'Chromosome': chrom_dict.name
            })
    elif (not genome_summaries.assemblies) or (not genomic_locations):
        print(f"No genomes found for organism: {organism_name}")

    return genomic_locations

In [4]:
# Walnut

get_chrom_locations('Juglans regia (English walnut)')

[{'RefSeq': 'NC_049901.1', 'Size': '45207397', 'Chromosome': '1'},
 {'RefSeq': 'NC_049902.1', 'Size': '37821870', 'Chromosome': '2'},
 {'RefSeq': 'NC_049903.1', 'Size': '35064427', 'Chromosome': '3'},
 {'RefSeq': 'NC_049904.1', 'Size': '34823025', 'Chromosome': '4'},
 {'RefSeq': 'NC_049905.1', 'Size': '22562875', 'Chromosome': '5'},
 {'RefSeq': 'NC_049906.1', 'Size': '39020271', 'Chromosome': '6'},
 {'RefSeq': 'NC_049907.1', 'Size': '52418484', 'Chromosome': '7'},
 {'RefSeq': 'NC_049908.1', 'Size': '30564197', 'Chromosome': '8'},
 {'RefSeq': 'NC_049909.1', 'Size': '24263475', 'Chromosome': '9'},
 {'RefSeq': 'NC_049910.1', 'Size': '37707155', 'Chromosome': '10'},
 {'RefSeq': 'NC_049911.1', 'Size': '37114715', 'Chromosome': '11'},
 {'RefSeq': 'NC_049912.1', 'Size': '31492331', 'Chromosome': '12'},
 {'RefSeq': 'NC_049913.1', 'Size': '39757759', 'Chromosome': '13'},
 {'RefSeq': 'NC_049914.1', 'Size': '28841373', 'Chromosome': '14'},
 {'RefSeq': 'NC_049915.1', 'Size': '20407330', 'Chromosom

##### GeneAPI

In [5]:
gene_api = GeneApi()
help(gene_api)

Help on GeneApi in module ncbi.datasets.openapi.api.gene_api object:

class GeneApi(builtins.object)
 |  GeneApi(api_client=None)
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator
 |  Ref: https://openapi-generator.tech
 |  
 |  Do not edit the class manually.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, api_client=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  download_gene_package(self, gene_ids, **kwargs)
 |      Get a gene dataset by gene ID  # noqa: E501
 |      
 |      Get a gene dataset including gene, transcript and protein fasta sequence, annotation and metadata by gene ID.  # noqa: E501
 |      This method makes a synchronous HTTP request by default. To make an
 |      asynchronous HTTP request, please pass async_req=True
 |      
 |      >>> thread = api.download_gene_package(gene_ids, async_req=True)
 |      >>> result = thread.get()
 |      
 |      Args:
 |          gene_ids ([int]): NCBI gene ids
 |      
 

In [7]:
walnut_accessions = get_chrom_locations('Juglans regia (English walnut)')
walnut_ref_seq = [chrom['RefSeq'] for chrom in walnut_accessions if chrom['RefSeq'] is not None]
# walnut_ref_seq

gene_api.gene_tax_name_query('Juglans regia (English walnut)')

{'sci_name_and_ids': [{'common_name': 'English walnut',
                       'sci_name': 'Juglans regia',
                       'tax_id': '51240'},
                      {'sci_name': 'Juglans microcarpa x Juglans regia',
                       'tax_id': '2249226'},
                      {'sci_name': 'Juglans sigillata x Juglans regia',
                       'tax_id': '1441050'},
                      {'sci_name': 'Juglans major x Juglans regia',
                       'tax_id': '1141591'},
                      {'sci_name': 'Roystonea regia', 'tax_id': '145709'},
                      {'sci_name': 'Drosera regia', 'tax_id': '4371'},
                      {'sci_name': 'Odontella regia', 'tax_id': '1335017'},
                      {'common_name': 'English oak',
                       'sci_name': 'Quercus robur',
                       'tax_id': '38942'},
                      {'common_name': 'black walnut',
                       'sci_name': 'Juglans nigra',
                       't

##### GenomeAPI

In [31]:
genome_api = GenomeApi()
help(genome_api)

Help on GenomeApi in module ncbi.datasets.openapi.api.genome_api object:

class GenomeApi(builtins.object)
 |  GenomeApi(api_client=None)
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator
 |  Ref: https://openapi-generator.tech
 |  
 |  Do not edit the class manually.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, api_client=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  assembly_descriptors_by_accessions(self, accessions, **kwargs)
 |      Get genome metadata by accession  # noqa: E501
 |      
 |      Get detailed metadata for assembled genomes by accession in a JSON output format.  # noqa: E501
 |      This method makes a synchronous HTTP request by default. To make an
 |      asynchronous HTTP request, please pass async_req=True
 |      
 |      >>> thread = api.assembly_descriptors_by_accessions(accessions, async_req=True)
 |      >>> result = thread.get()
 |      
 |      Args:
 |          accessions ([str]):
 |      


In [33]:
client = ApiClient()
genome_api = GenomeApi(client)

# Search for genome assemblies by organism name
genome_summaries = genome_api.assembly_descriptors_by_taxon('Juglans regia (English walnut)')
print(genome_summaries.assemblies)

[{'assembly': {'annotation_metadata': {'busco': {'busco_lineage': 'eudicots_odb10',
                                                'busco_ver': '4.0.2',
                                                'complete': 0.99269134,
                                                'duplicated': 0.091573514,
                                                'fragmented': 0.0017196905,
                                                'missing': 0.005588994,
                                                'single_copy': 0.9011178,
                                                'total_count': '2326'},
                                      'file': [{'estimated_size': '11438918',
                                                'type': 'GENOME_GFF'},
                                               {'estimated_size': '261191555',
                                                'type': 'GENOME_GBFF'},
                                               {'estimated_size': '23490972',
                           

#### Reading Data via Entrez API (affiliated with NCBI)

In [7]:
!pip install Bio

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------------------------------------- -- 2.6/2.8 MB 13.7 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 10.1 MB/s eta 0:00:00


In [8]:
from Bio import Entrez

In [10]:
Entrez.email = "adamhetherwick@gmail.com"
handle = Entrez.egquery(term="Juglans regia (English walnut)")
record = Entrez.read(handle)

URLError: <urlopen error [Errno 11001] getaddrinfo failed>